In [ ]:
using Flight.FlightCore.Systems
using Flight.FlightCore.Plotting

using Flight.FlightPhysics.Geodesy
using Flight.FlightPhysics.Kinematics

using Flight.FlightComponents.Aircraft

using Flight.FlightAircraft.C172R

using ControlSystems

In [ ]:
ac = Cessna172Rv0(NED()) |> System #linearization requires NED kinematics
ac.u.airframe.pld.m_pilot = 75
ac.u.airframe.pld.m_copilot = 75
ac.u.airframe.pld.m_lpass = 0
ac.u.airframe.pld.m_rpass = 0
ac.u.airframe.pld.m_baggage = 0

trim_params = C172Rv0.TrimParameters(
    loc = LatLon(),
    h = HOrth(1000),
    ψ_nb = 0.0,
    TAS = 40.0,
    γ_wOb_n = 0.0,
    ψ_lb_dot = 0.00,
    θ_lb_dot = 0.0,
    β_a = 0.0,
    fuel = 1,
    mixture = 0.5,
    flaps = 0.0)

lm = Aircraft.linearize!(ac; trim_params)

### Longitudinal Dynamics

In [ ]:
long_dyn = filter(lm;
    u = (:elevator, :throttle),
    x = (:v_x, :v_z, :θ, :q, :α_filt, :ω_eng),
    y = (:q, :θ, :α, :TAS, :f_x, :f_z, :ω_eng))

long_dyn_ss = long_dyn |> ss
long_dyn_tf = long_dyn_ss |> tf

#we can reduce the model further for CAS by omitting omega_eng and throttle
pitch_dyn = filter(lm; u = (:elevator,),
    x = (:v_x, :v_z, :θ, :q, :α_filt),
    y = (:q, :θ, :α, :TAS))

pitch_dyn_ss = pitch_dyn |> ss
pitch_dyn_tf = pitch_dyn_ss |> tf


In [ ]:
#extract the elevator to q transfer function
e2q = long_dyn_tf[1,1]
@show zpk(e2q)

#the open loop TF is type 0, but with a zero almost at the origin, so in
#practice it has nearly derivative action. we can see that the steady state
#error for a step input is nearly 1, that is, the output falls back almost to
#zero
err_e2q = 1/(1+e2q)
@show zpk(err_e2q)
@show err_e2q(0)

#therefore, although adding a pure integrator will in theory yield a type 1
#system (and zero steady state error for step inputs), in practice (for
#reasonable time windows) it will still behave as a type 0. so, in addition to
#this integrator we can expect to require some more integral action in the
#feedforward path for satisfactory response.


In [ ]:
#we can start by trying a PID in series with the pure integrator previously
#indicated. rather than aiming for minimum steady state error (since the outward
#θ loop doesn't really care for that), we want reasonable stability margins

# k_p = 10
# k_i = 20
# k_d = 0.5
k_p = 4
k_i = 35
k_d = 0.6
q_comp = series(k_p + k_i * tf(1, [1,0]) + k_d * tf([1, 0], [0.05, 1]), tf(1, [1, 0]))
    
e2q_c = series(q_comp, e2q)
e2q_c = minreal(e2q_c, 1e-3)
@show zpk(e2q_c)

#error transfer function (setpoint - plant output)
e2q_c_err = 1/(1+e2q_c)
#the steady state error for a given test input is the final value of the ETF for
#that input. applying the final value theorem, the final value for a step input
#is found simply by settng s=0. therefore:
@show ss_err_step = e2q_c_err(0)

#and for a ramp input:
@show ss_err_ramp = (tf(1, [1, 0]) * e2q_c_err)(0)

marginplot(e2q_c)


In [ ]:
cl = feedback(e2q_c, 1)
@show zpk(cl)
@show poles(cl)
# bodeplot(cl)
step(cl, 10) |> plot

Now we have a SISO compensator design for q, let's use it to close the loop in the MIMO plant. The long_dyn MIMO system has two inputs, elevator input u_e and throttle input u_t. Our SISO compensator receives q_err as an input and outputs u_e. So the MIMO version of our SISO compensator, which will be placed upstream of the MIMO plant, must receive q_err and u_t as inputs and output u_e and u_t. Therefore, it must apply the SISO compensator to q_err to obtain u_e and let u_t pass through unchanged to the plant by applying an unit gain to it. This is achieved as follows:

In [ ]:
#create a diagonal MIMO system from the pitch rate compensator and a unit
#transfer function
q_comp_MIMO = append(q_comp, tf(1)) |> ss #and convert to ss before concatenation

In [ ]:
q_comp_long_dyn = series(q_comp_MIMO, long_dyn_ss) #equivalent to long_dyn_ss * q_comp_MIMO (in the product notation, the second system goes first)

The setpoint vector to be applied to the closed-loop MIMO system is r = [q_cmd, u_t]. The input to the MIMO compensator must be e = [q_err, u_t] = r - f = [q_cmd, u_t] - [q, 0]. So we need a matrix gain that premultiplies the 7-component output vector of long_dyn, and produces a 2-component vector [q, 0]. This will be a 2x7 matrix with all entries set to zero except [1, 1] (the nonzero output goes in the first component, and q is the first element in long_dyn's output vector)

In [ ]:
K_q = zeros(2, 7)
K_q[1,1] = 1
K_q_ss = ss(K_q) #creates a ss with no dynamics and K_select as feedthrough

#now create a feedback loop with K in the feedback path
q_comp_long_dyn_cl = feedback(q_comp_long_dyn, K_q_ss)

#this is similar, but NOT equivalent: it uses unit feedback, which means that
#the whole 7 component output vector from q_comp_long_dyn is fed back, and then
#multiplied by K_ss (and so reduced to 2 components) in the feedforward path
#before entering the compensator. this yields a 7 input closed loop system, of
#which only the two first inputs actually do anything (the others get discarded
# when they pass through K) q_comp_long_dyn_cl = feedback(series(K_ss,
# q_comp_long_dyn))

e2q_cl_MIMO = (q_comp_long_dyn_cl |> tf)[1,1]

#make sure the zpk data matches that of the SISO closed loop TF
@show zpk(e2q_cl_MIMO)

#check step response
step(e2q_cl_MIMO, 10) |> plot


In [ ]:
#let's see what we could achieve if we had an actuator in the loop with a 5Hz time constant
k_p = 4
k_i = 8
k_d = 0.3
τ_d = 0.02
q_comp = tf(1, [1, 0]) * (k_p + k_i * tf(1, [1,0]) + k_d * tf([1, 0], [τ_d, 1]))
act = tf(1, [0.2, 1])
    
e2q_c = q_comp * act * e2q
e2q_c = minreal(e2q_c, 1e-3)
@show zpk(e2q_c)
e2q_c_err = 1/(1+e2q_c)
#the steady state error for a given test input is the final value of the ETF for
#that input. applying the final value theorem, the final value for a step input
#is found simply by settng s=0. therefore:
@show ss_err_step = e2q_c_err(0)


marginplot(e2q_c)
# rlocusplot(e2q_c)
# nyquistplot(e2q_c)
# bodeplot(act)


In [ ]:
cl = feedback(e2q_c, 1)
step(cl, 10) |> plot

### $\Theta$ Control

Returning now to the longitudinal dynamics with q feedback, we have a MIMO plant with two inputs, q_cmd and u_t, and the same initial seven outputs (q, θ...). We can now retrieve the open loop response from the first input (q_cmd) to the second output ($\theta$) as:

In [ ]:
#the previous closed loop MIMO plant with q_cmd as its first input is our
#starting plant for this design
q2θ = q_comp_long_dyn_cl[2, 1] #no need to transform to TF first
@show zpk(q2θ)

In [ ]:
err_q2θ = 1/(1+tf(q2θ)) 
@show err_q2θ(0) #steady state error for a step input
#we can see this is essentially a type 1 transfer function. we can achieve zero
#steady state error for a step input just with proportional control

In [ ]:
k_θ = 2.8
θ_cmp = ss(k_θ)
    
c_q2θ = series(θ_cmp, q2θ)
c_q2θ = minreal(c_q2θ, 1e-3)
@show zpk(c_q2θ)

marginplot(c_q2θ)

In [ ]:
cl = feedback(c_q2θ, 1)
step(cl, 10) |> plot

The design is satisfactory, let's build the MIMO system as before.

In [ ]:
#create a diagonal MIMO system from the theta compensator and a unit transfer
#function
θ_cmp_MIMO = append(θ_cmp, tf(1)) |> ss #and convert to ss before concatenation
θ_cmp_q_cmp_long_dyn = series(θ_cmp_MIMO, q_comp_long_dyn_cl)

In [ ]:
K_θ = zeros(2, 7)
K_θ[1,2] = 1 #extracts the second output vector component and puts it in the first component
K_θ_ss = ss(K_θ) #creates a ss with no dynamics and K_select as feedthrough

#now create a feedback loop with K in the feedback path
θ_cmp_q_cmp_long_dyn_cl = feedback(θ_cmp_q_cmp_long_dyn, K_θ_ss)

θ_cl_MIMO = θ_cmp_q_cmp_long_dyn_cl[2,1]

#make sure the zpk data matches that of the SISO closed loop TF
@show zpk(θ_cl_MIMO)

#check step response
step(θ_cl_MIMO, 10) |> plot
